### setup

In [140]:
import requests
import json

url_stub = "http://192.168.99.108:8000/api/v1" # or https://validation-server-stg.urban.org/api/v1/

/api-auth-token/

In [141]:
user_account = {
    "username": "admin",  # your username
    "password": "admin"   # your password
}

r = requests.post(f"http://192.168.99.108:8000/api/api-token-auth/", data=user_account)


In [142]:
print(r.status_code)
print(r.reason)
print(r.text)

token = r.json()["token"]
print(token)

headers = {"Authorization": f"Token {token}"}


200
OK
{"token":"29f7c47590b3dca77a1c1fff3a8c2e6a04d4acaf"}
29f7c47590b3dca77a1c1fff3a8c2e6a04d4acaf


### /api/v1/command/

#### post a new command

* `POST` upon new file upload

In [145]:
sanitized_command_input = {
        "analysis_query":"fakequery1",
        "transformation_query":"alsofake1"
    }

payload = {
    "command_type": 2,
    "command_name" : "test-5", 
    "sanitized_command_input": json.dumps(sanitized_command_input)
}

r = requests.post(f"{url_stub}/command/", headers=headers, data=payload)

In [146]:
print(r.status_code)
print(r.reason)
print(r.text)

for key, value in r.json().items():
    print(key, value)

201
Created
{"command_id":4,"researcher_id":1,"command_name":"test-5","command_type":2,"sanitized_command_input":{"analysis_query":"fakequery1","transformation_query":"alsofake1"}}
command_id 4
researcher_id 1
command_name test-5
command_type 2
sanitized_command_input {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}


#### get a command

In [147]:
command_id = r.json()["command_id"]
r = requests.get(f"{url_stub}/command/{command_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
researcher_id 1
command_name test-5
command_type 2
sanitized_command_input {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}


#### rename a command

* The mockup allows you to rename a command

In [ ]:
payload = {
    "command_name" : "new_command_name", 
}
command_id = r.json()["command_id"]
r = requests.patch(f"{url_stub}/command/{command_id}/", headers=headers, data=payload)
print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

#### get list of commands

In [148]:
r2 = requests.get(f"{url_stub}/command/", headers=headers)

print(r2.status_code)
print(r2.reason)
for key, value in r2.json().items():
    print(key, value)

200
OK
count 4
next None
previous None
results [{'command_id': 4, 'researcher_id': 1, 'command_name': 'test-5', 'command_type': 2, 'sanitized_command_input': {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}}, {'command_id': 3, 'researcher_id': 1, 'command_name': 'test-4', 'command_type': 2, 'sanitized_command_input': {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}}, {'command_id': 2, 'researcher_id': 1, 'command_name': 'test-3', 'command_type': 2, 'sanitized_command_input': {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}}, {'command_id': 1, 'researcher_id': 1, 'command_name': 'test-9', 'command_type': 2, 'sanitized_command_input': {'analysis_query': 'fakequery1', 'transformation_query': 'alsofake1'}}]


### /api/v1/synthetic-data-run/

#### get a synthetic data run

In [149]:
r = requests.get(f"{url_stub}/synthetic-data-run/1/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 1
run_id 1
epsilon 1.00
date_time_run_submitted 2021-08-17T15:13:37+0000


#### get a synthetic data run querying by command_id

In [150]:
r = requests.get(f"{url_stub}/synthetic-data-run?command_id={command_id}", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
count 1
next None
previous None
results [{'command_id': 4, 'run_id': 4, 'epsilon': '1.00', 'date_time_run_submitted': '2021-08-17T17:28:46+0000'}]


### /api/v1/synthetic-data-result/

#### get a synthetic data result

* What is the best way to signal if a run failed?

In [ ]:
r = requests.get(f"{url_stub}/synthetic-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/confidential-data-run

#### post a confidential data run

* Will actually be multiple `POST` requests

In [153]:
payload = {
    "command_id": command_id, # from the POST to command
    "epsilon": 1.00, # will always be 1 for POC
}

r = requests.post(f"{url_stub}/confidential-data-run/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)

for key, value in r.json().items():
    print(key, value)

201
Created
command_id 4
run_id 3
epsilon 1.00
date_time_run_submitted 2021-08-17T17:36:38+0000


#### get a confidential data run

In [154]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-run/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
command_id 4
run_id 3
epsilon 1.00
date_time_run_submitted 2021-08-17T17:36:38+0000


#### get a list of confidential data runs querying by command_id

In [155]:
r = requests.get(f"{url_stub}/confidential-data-run?command_id={command_id}", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

200
OK
count 3
next None
previous None
results [{'command_id': 4, 'run_id': 1, 'epsilon': '1.00', 'date_time_run_submitted': '2021-08-17T17:36:19+0000'}, {'command_id': 4, 'run_id': 2, 'epsilon': '1.00', 'date_time_run_submitted': '2021-08-17T17:36:33+0000'}, {'command_id': 4, 'run_id': 3, 'epsilon': '1.00', 'date_time_run_submitted': '2021-08-17T17:36:38+0000'}]


### /api/v1/confidential-data-result/

#### get a confidential data result

In [ ]:
run_id = r.json()["run_id"]
r = requests.get(f"{url_stub}/confidential-data-result/{run_id}/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

### /api/v1/public-user-budget/

#### get user budget

In [ ]:
r = requests.get(f"{url_stub}/public-use-budget/", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### update user budget

In [ ]:
payload = {'privacy_budget_used': '9'}
researcher_id = 1
r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### don't go over budget

In [ ]:
payload = {'privacy_budget_used': '1000'}

r = requests.patch(f"{url_stub}/public-use-budget/{researcher_id}/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
print(r.text)